<a href="https://colab.research.google.com/github/sahil3Vedi/Enron-Email-Classifier/blob/master/Context_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import re
import os
import numpy as np
import random
from numpy import dot
from numpy.linalg import norm

In [7]:
ENRONFOLDER = 'drive/My Drive/Enron Spam/'
ENRONNOSPAMDATASET = 'drive/My Drive/Enron Spam/ham'
FEATURES_LOC = 'drive/My Drive/Enron Spam/featureVectors.npy'

feature_vectors = np.load(FEATURES_LOC)

#1) Affirmative Context:
#(a) Thank You, I will look into it
#(b) Thank you for informing
#
#2) Proposal Context:
#(a) Sure, we can proceed
#(b) I am not sure we can proceed
# 
#3) Follow up Context:
#(a) Okay, I will follow up as required
#(b) Unfortunately, I am unable to follow up
#
#Invite context:
#(a) Thank You, I will be there
#(b) Thank you, but I won't be able to make it

#Each context has a flag and an associated context vector(centroid) (just like formal/informal classification)

affirmative_flag = 0
proposal_flag = 0
followup_flag = 0
invite_flag = 0

affirmative_centroid = np.zeros(1000)
proposal_centroid = np.zeros(1000)
followup_centroid = np.zeros(1000)
invite_centroid = np.zeros(1000)

learning_rate = 0.1

annotation_indices = []
annotations = []

print("Feature Vectors loaded! Dimensions of the feature vectors matrix: " + str(np.shape(feature_vectors)))

Feature Vectors loaded! Dimensions of the feature vectors matrix: (3681, 1000)


In [0]:
def getLabel(mailIndex):
  file_location = os.listdir(ENRONNOSPAMDATASET)[mailIndex]
  FILENAME = os.path.join(ENRONNOSPAMDATASET, file_location)
  with open(FILENAME, 'r',encoding='utf-8',errors='ignore') as myfile:
    data = myfile.read()
    print("Message Contents: ")
    print(data)
    print("\n Is the context of the displayed displayed message affirmative[0] or proposal[1] or followup[2] or invite[3] ?")
    val = int(input("Enter your value: "))
    return val

def centroid_update(anno_index, label_value):
  
  global affirmative_flag
  global proposal_flag
  global followup_flag
  global invite_flag

  global affirmative_centroid
  global proposal_centroid
  global followup_centroid
  global invite_centroid

  global learning_rate

  fvector = feature_vectors[anno_index]

  a_flag = affirmative_flag
  p_flag = proposal_flag
  f_flag = followup_flag
  i_flag = invite_flag

  if(label_value==0):
    if(a_flag == 0):
      affirmative_flag = 1
      affirmative_centroid = fvector
    else:
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.add(mult, affirmative_centroid)
    if(p_flag == 1):
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.subtract(proposal_centroid, mult)
    if(f_flag == 1):
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.subtract(followup_centroid, mult)
    if(i_flag == 1):
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.subtract(invite_centroid, mult)
    
  
  elif(label_value==1):
    if(p_flag == 0):
      proposal_flag = 1
      proposal_centroid = fvector
    else:
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.add(mult, proposal_centroid)
    if(a_flag == 1):
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.subtract(affirmative_centroid, mult)
    if(f_flag == 1):
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.subtract(followup_centroid, mult)
    if(i_flag == 1):
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.subtract(invite_centroid, mult)

  elif(label_value==2):
    if(f_flag == 0):
      followup_flag = 1
      followup_centroid = fvector
    else:
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.add(mult, followup_centroid)
    if(a_flag == 1):
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.subtract(affirmative_centroid, mult)
    if(p_flag == 1):
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.subtract(proposal_centroid, mult)
    if(i_flag == 1):
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.subtract(invite_centroid, mult)

  elif(label_value==3):
    if(i_flag == 0):
      invite_flag = 1
      invite_centroid = fvector
    else:
      diff = np.subtract(fvector, invite_centroid)
      mult = np.multiply(diff, learning_rate)
      invite_centroid = np.add(mult, invite_centroid)
    if(a_flag == 1):
      diff = np.subtract(fvector, affirmative_centroid)
      mult = np.multiply(diff, learning_rate)
      affirmative_centroid = np.subtract(affirmative_centroid, mult)
    if(p_flag == 1):
      diff = np.subtract(fvector, proposal_centroid)
      mult = np.multiply(diff, learning_rate)
      proposal_centroid = np.subtract(proposal_centroid, mult)
    if(f_flag == 1):
      diff = np.subtract(fvector, followup_centroid)
      mult = np.multiply(diff, learning_rate)
      followup_centroid = np.subtract(followup_centroid, mult)

  #if((f_flag == 0) and (label_value==0)):
    #formal_flag = 1
    #formal_centroid = fvector
  #elif((if_flag == 0) and (label_value==1)):
    #informal_flag = 1
    #informal_centroid = fvector
    #print(fvector)
    #print(informal_centroid)
  #elif((f_flag ==1) and (label_value==0)):
    #diff = np.subtract(fvector, formal_centroid)
    #mult = np.multiply(diff, learning_rate)
    #formal_centroid = np.add(mult,formal_centroid)
  #elif((if_flag ==1) and (label_value==1)):
    #diff = np.subtract(fvector, informal_centroid)
    #mult = np.multiply(diff, learning_rate)
    #informal_centroid = np.add(mult,informal_centroid)
  else:
    print("Input Error")

  print("Updated Context Centroids!")
  #print("Affirmative:" + str(affirmative_centroid))
  #print("Proposal:" + str(proposal_centroid))
  #print("Followup:" + str(followup_centroid))
  #print("Invite:" + str(invite_centroid))

In [0]:
def generateIndex(max_val):
  affirmatives = 0
  proposals = 0
  followups = 0
  invites = 0
  for j in annotations:
    if j == 0:
      affirmatives += 1
    elif j == 1:
      proposals += 1
    elif j == 2:
      followups += 1
    elif j == 3:
      invites += 1
    else:
      print("Annotation Error, kindly check code and restart.. Annotate carefully!")
  cc_array = [affirmatives, proposals, followups, invites]
  centroids = [affirmative_centroid, proposal_centroid, followup_centroid, invite_centroid]

  #Check if any two elements in the list are equal
  seen = []
  repeat_flag = 0
  for each_number in cc_array:
    if each_number in seen:
      repeat_flag = 1
    else:
      seen.append(each_number)

  #Generate Random Index if elements are repeated or any element is 0
  if ((0 in cc_array) or (repeat_flag==1)):
    genIndex = random.randrange(max_val)
  #Or generate an index that is most similar to the least featured centroid
  else:
    #generating 10 random indices and storing their feature vectors
    vector_indices = []
    for i in range(10):
      vector_indices.append(random.randrange(max_val))
    index_features = [feature_vectors[x] for x in vector_indices]
    #finding least annotated centroid
    minInd = cc_array.index(min(cc_array))
    rarest_centroid = centroids[minInd]
    cos_sims = [cosineSim(y,rarest_centroid) for y in index_features]
    print(cos_sims)
    #choosing feature index with highest similarity with the least featured centroid
    genInd = cos_sims.index(max(cos_sims))
    genIndex = vector_indices[genInd]
  return genIndex

In [0]:
def annotateLabels(num_labels):
  for i in range(num_labels):
    anno_index = generateIndex(3100)
    annotation_indices.append(anno_index)
    label_value = getLabel(anno_index)
    annotations.append(label_value)
    centroid_update(anno_index, label_value)


In [11]:
annotateLabels(10)


Message Contents: 
Subject: # 9760
tried to get fancy with your address and it came back to me !
- - - - - - - - - - - - - - - - - - - - - - forwarded by lauri a allen / hol / aepin on 10 / 25 / 2001 01 : 11 pm - - - - - - - - - - - - - - - - - - - - - - - - - - -
to : daren . farmer @ enron . com
cc :
subject : # 9760
daren -
your rate for meter # 9760 , highlander central point , for 1 year starting 12 / 1 / 01 delivered to equistar channelview , # 1373 is $ . 07 / mm . that price expires in 1 week , on november 1 , 2001 . let me know if you need me to refresh after that time . thanks !

 Is the context of the displayed displayed message affirmative[0] or proposal[1] or followup[2] or invite[3] ?
Enter your value: 2
Updated Context Centroids!
Message Contents: 
Subject: cleburne
cleburne should be it ' s own desk . rick hill should be set up as trader / head
of desk . let me know if you have more questions .
dave

 Is the context of the displayed displayed message affirmative[0] or p

In [12]:
  print("Affirmative:" + str(affirmative_centroid))
  print("Proposal:" + str(proposal_centroid))
  print("Followup:" + str(followup_centroid))
  print("Invite:" + str(invite_centroid))

Affirmative:[-0.206711   1.0978021 -0.2175921 -0.197811   1.0978021  0.
 -0.4343031 -0.3254031  1.0978021  1.1056131 -0.189     -0.3353031
 -0.1079     1.1878021 -0.2174921  0.        -0.216711  -0.216711
  1.2056131 -0.2079    -0.216711  -0.2174921  1.4343031  1.5243031
  0.12079    0.        -0.1089    -0.107811   0.         0.
 -0.216711  -0.315711  -0.1089     0.         0.111979  -0.09
 -0.0089    -0.2264031 -0.1185921 -0.1089     0.        -0.189
 -0.2274921 -0.09      -0.09      -0.107811  -0.206811  -0.1089
 -0.1089     0.02979    0.         0.1        0.         0.
 -0.2085921 -0.1089     0.        -0.1185921 -0.1185921  0.
 -0.1185921 -0.107811  -0.2079    -0.1089    -0.2274921  0.
  0.         0.        -0.007811   0.         0.        -0.106811
  0.        -0.1185921 -0.1089    -0.1185921  0.         0.1
 -0.099     -0.107811  -0.1185921  0.         0.        -0.1089
 -0.107811  -0.09       0.         0.        -0.09       1.4045131
  1.4045131  1.4045131 -0.216711   0.    

In [13]:
print(annotation_indices)
print(annotations)

[2541, 2646, 966, 2178, 1269, 273, 2643, 937, 639, 1056]
[2, 2, 0, 2, 1, 0, 3, 1, 1, 0]


In [0]:
#Storing annotation indices and annotations for testing
annoFile = 'drive/My Drive/Enron Spam/context_annotations.npy'
annoIndFile = 'drive/My Drive/Enron Spam/context_annotation_indices.npy'
#Storing context centroid vectors
affirmativeFile = 'drive/My Drive/Enron Spam/affirmative_vector.npy'
proposalFile = 'drive/My Drive/Enron Spam/proposal_vector.npy'
followupFile = 'drive/My Drive/Enron Spam/followup_vector.npy'
inviteFile = 'drive/My Drive/Enron Spam/invite_vector.npy'

np.save(annoFile, annotations)
np.save(annoIndFile, annotation_indices)

np.save(affirmativeFile, affirmative_centroid)
np.save(proposalFile, proposal_centroid)
np.save(followupFile, followup_centroid)
np.save(inviteFile, invite_centroid)